In [ ]:

from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as uReq
import csv


def main():
    url_Tickers = 'https://money.cnn.com/data/hotstocks/'
    firstTable_diction = dict()  # Hold Ticker name & Company name
    secondTable_diction = dict()  # Hold Ticker name & Company name
    thirdTable_diction = dict()  # Hold Ticker name & Company name
    uClient = uReq(url_Tickers)
    page_html = uClient.read()   # read all the page
    uClient.close                  # Close after finish reading
    page_html = soup(page_html,'html.parser') # Parser page to Html Languge
    h3_list = list() # Store h3 values in a List


# Loop through the html codes
    for i in range(0, len(page_html) - 1):
         most_Activity = page_html.find(id='wsod_hotStocks').find_all('h3')[i].string  # Extract  MostActives , Gainers , Losers  Words
         h3_list.append(most_Activity)

    search_location =0
    while search_location<3:
        tr_table = page_html.find_all(class_='wsod_dataTable wsod_dataTableBigAlt')[search_location]  # this line is take each table between the 3 tables
        parameter =1
        while True:
            s=tr_table.find_all('tr')[parameter]        # take first tr in the first table start from 1 not 0
            td_table =s.find_all('td')[0]       # take first td inside tr which inside table
            link_a = td_table.find_all('a')[0].text   # take the first link text inside first td
            span_words =td_table.find_all('span')[0]    # Extract span
            dd = span_words.contents[0]
            if search_location ==0:
                firstTable_diction[link_a] = dd
                parameter =parameter+1
                if  parameter ==11:
                    search_location =search_location+1
                    break
            if search_location == 1:
                secondTable_diction[link_a] = dd
                parameter = parameter + 1
                if parameter == 11:
                    search_location = search_location + 1
                    break
            if search_location ==2:
                thirdTable_diction[link_a] = dd
                parameter = parameter + 1
                if parameter == 11:
                    search_location = search_location + 1
                    break



    # Get All Keys and Values of Each Diction (Contains Ticker name , Company Name)
    ticker_MostActivity_part = list(firstTable_diction.keys())
    ticker_MostActivity_companyNames = list(firstTable_diction.values())

    ticker_Gainers_part = list(secondTable_diction.keys())
    ticker_Gainers_companyNames = list(secondTable_diction.values())

    ticker_losers_part = list(thirdTable_diction.keys())
    ticker_losers_companyNames = list(thirdTable_diction.values())



    most_Actitvity_dict =dict()  # Hold Key as Ticker name Value Ticker Details from 2nd Page
    Gainer_dict =dict()         # Hold Key as Ticker name Value Ticker Details from 2nd Page
    losers_dict =dict()          # Hold Key as Ticker name Value Ticker Details from 2nd Page


    # tickers will loop through all keys "Tickers" and open link to 2nd page of each Ticker and read all Stock info and store it in ""secondPage_details""
    #for tickers in range(len(list_all_Tickers)):  # has 3 indexes 0,1,2 each index has sub Tickers [0] = CCL,GE,BAC..  [1] = OXY,FTI,NOV... [2]=SRE,RCL,HBI....
    for tickers in ticker_MostActivity_part:  # this will Check only Tickers of Most Activity
            secondPage_details = dict()

            # Must send each Ticker to check its details in 2nd Page
            url = 'https://money.cnn.com/quote/quote.html?symb=' + tickers
            uCCl = uReq(url)
            page_html_CCl = uCCl.read()
            uCCl.close
            page_html_CCl = soup(page_html_CCl, 'html.parser')
            collection_table = page_html_CCl.find_all(class_='wsod_dataTable wsod_dataTableBig')[0]
            for i in range(len(collection_table)):
                tr = collection_table.find_all('tr')[i]
                td = tr.find_all('td')[0].text
                td_span = tr.find_all('td')[1].text
                secondPage_details[td] = td_span
            most_Actitvity_dict[tickers] = secondPage_details
    most_gategory =h3_list[0]
    most_Activity_gategory =dict()
    most_Activity_gategory[most_gategory] =firstTable_diction


    list77 = list()
    for kk, mm in most_Actitvity_dict.items():
        for tt, pp in mm.items():
            list77.append(pp)

    tick_mostActivity_value = ''
    comapnyName = ''
    Previous_close = ''
    Today_s_open = ''
    Day_s_range = ''
    Volume = ''
    Average_volume = ''
    Market_cap = ''

    MostActivity = h3_list[0]
    dimintion = len(list77)
    counter2 = 0

    csv_file = open('stocks.csv', 'w', newline='')
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['Categories', 'Tickers symbols', 'Stock name', 'Previous close', 'Today’s open', 'Day’s range', 'Volume','Average volume (3 months)', 'Market cap'])



    # Start writting info into CSV file Row by Row
    for i in range(0, len(ticker_MostActivity_part)):
        tick_mostActivity_value = ticker_MostActivity_part[i]
        comapnyName = ticker_MostActivity_companyNames[i]
        while counter2 < dimintion:
            Previous_close = list77[counter2]
            Today_s_open = list77[counter2 + 1]
            Day_s_range = list77[counter2 + 2]
            Volume = list77[counter2 + 3]
            Average_volume = list77[counter2 + 4]
            Market_cap = list77[counter2 + 5]
            counter2 = counter2 + 6
            break

        csv_writer.writerow([MostActivity,tick_mostActivity_value,comapnyName,Previous_close,Today_s_open,Day_s_range,Volume,Average_volume,Market_cap])

    csv_file.close()


    print('\n')


    for tickers in ticker_Gainers_part:  # this will Check only Tickers of Gainer
            secondPage_details = dict()

            # Must send each Ticker to check its details in 2nd Page
            url = 'https://money.cnn.com/quote/quote.html?symb=' + tickers
            uCCl = uReq(url)
            page_html_CCl = uCCl.read()
            uCCl.close
            page_html_CCl = soup(page_html_CCl, 'html.parser')
            collection_table = page_html_CCl.find_all(class_='wsod_dataTable wsod_dataTableBig')[0]
            for i in range(len(collection_table)):
                tr = collection_table.find_all('tr')[i]
                td = tr.find_all('td')[0].text
                td_span = tr.find_all('td')[1].text
                secondPage_details[td] = td_span
            Gainer_dict[tickers] = secondPage_details
    gainer_gategory = h3_list[1]
    gainer_activity_gategory = dict()
    gainer_activity_gategory[gainer_gategory] = secondTable_diction


    list77 = list()
    for kk, mm in Gainer_dict.items():
        for tt, pp in mm.items():
            list77.append(pp)

    tick_gainer_value = ''
    comapnyName = ''
    Previous_close = ''
    Today_s_open = ''
    Day_s_range = ''
    Volume = ''
    Average_volume = ''
    Market_cap = ''

    gainer = h3_list[1]
    dimintion = len(list77)
    counter2 = 0

    csv_file = open('stocks.csv', 'a', newline='')
    csv_writer = csv.writer(csv_file)

    # Start append info into CSV file Row by Row
    for i in range(0, len(ticker_Gainers_part)):
        tick_gainer_value = ticker_Gainers_part[i]
        comapnyName = ticker_Gainers_companyNames[i]
        while counter2 < dimintion:
            Previous_close = list77[counter2]
            Today_s_open = list77[counter2 + 1]
            Day_s_range = list77[counter2 + 2]
            Volume = list77[counter2 + 3]
            Average_volume = list77[counter2 + 4]
            Market_cap = list77[counter2 + 5]
            counter2 = counter2 + 6
            break
        csv_writer.writerow([gainer, tick_gainer_value, comapnyName, Previous_close, Today_s_open, Day_s_range, Volume,Average_volume, Market_cap])

    csv_file.close()




    for tickers in ticker_losers_part:  # this will Check only Tickers of Gainer
            secondPage_details = dict()

            # Must send each Ticker to check its details in 2nd Page
            url = 'https://money.cnn.com/quote/quote.html?symb=' + tickers
            uCCl = uReq(url)
            page_html_CCl = uCCl.read()
            uCCl.close
            page_html_CCl = soup(page_html_CCl, 'html.parser')
            collection_table = page_html_CCl.find_all(class_='wsod_dataTable wsod_dataTableBig')[0]
            for i in range(len(collection_table)):
                tr = collection_table.find_all('tr')[i]
                td = tr.find_all('td')[0].text
                td_span = tr.find_all('td')[1].text
                secondPage_details[td] = td_span
            losers_dict[tickers] = secondPage_details
    loser_gategory = h3_list[2]
    loser_activity_gategory = dict()
    loser_activity_gategory[loser_gategory] = thirdTable_diction


    list77 = list()
    for kk,mm in losers_dict.items():
        for tt, pp in mm.items():
            list77.append(pp)

    tick_loser_value =''
    comapnyName=''
    Previous_close =''
    Today_s_open =''
    Day_s_range= ''
    Volume =''
    Average_volume =''
    Market_cap =''

    losers=h3_list[2]
    dimintion = len(list77)  # length = 60
    counter2 = 0

    csv_file = open('stocks.csv', 'a', newline='')
    csv_writer = csv.writer(csv_file)

    # Start append info into CSV file Row by Row
    for i in range(0,len(ticker_losers_part)):
            tick_loser_value = ticker_losers_part[i]
            comapnyName = ticker_losers_companyNames[i]
            while counter2<dimintion:
                Previous_close = list77[counter2]
                Today_s_open=list77[counter2+1]
                Day_s_range=list77[counter2+2]
                Volume=list77[counter2+3]
                Average_volume=list77[counter2+4]
                Market_cap=list77[counter2+5]
                counter2 =counter2+6
                break
            csv_writer.writerow([losers, tick_loser_value, comapnyName, Previous_close, Today_s_open, Day_s_range, Volume,Average_volume, Market_cap])

    csv_file.close()





    check_value_in_lists = 0
    print("Which stock are you interested in: ")

    # Append All Ticker Symbol together
    all_Ticker_Symbols = list()
    all_Ticker_Symbols.append(ticker_MostActivity_part)
    all_Ticker_Symbols.append(ticker_Gainers_part)
    all_Ticker_Symbols.append(ticker_losers_part)
    stop= True


    # Print a list of Ticker Symbols and Company name to User of All Categories
    while stop:
        print()
        for i in range(0, len(h3_list)):
            if i == 0:
                print(h3_list[i], ':')
                for k, j in firstTable_diction.items():  # item will return key,value of each part of Dictionary
                    print(k, j)  # i will print key , j will print values
            print('')
            if i == 1:
                print(h3_list[i], ':')
                for k, j in secondTable_diction.items():  # item will return key,value of each part of Dictionary
                    print(k, j)  # i will print key , j will print values
            print('')
            if i == 2:
                print(h3_list[i], ':')

                for k, j in thirdTable_diction.items():  # item will return key,value of each part of Dictionary
                    print(k, j)  # i will print key , j will print values
        print('\n')



        user_input = input("User inputs: ")

    # loop through all Ticket Symbols and check matching with user input. If found stop searching and print result by calling print_result funtion
        flag = True
        counter =0
        while flag:
            if counter == 3:
                break
            for j in all_Ticker_Symbols[counter]:
                if user_input == j:
                    check_value_in_lists = check_value_in_lists + 1
                    print_result(user_input)
                    flag =False
                    break
            if flag == False:
                break
            counter = counter+1


        if check_value_in_lists == 0:
            print("The Company You entered Not Available in Stock, Enter Correct Company!")

        user_input = input("Finish extracting by Enter N Or Y to Continue: ")
        while True:
            if user_input.lower() == 'n':
                stop =False
                break
            elif user_input.lower() == 'y':
                break

            else:
                user_input = input("Finish extracting by Enter N Or Y to Continue: ")




"""
This funtion is take input user
open CSV file
searching through the Rows for specific information 
if found stop searching & print results
"""
def print_result(user_input):

    try:
        file_handler = open('stocks.csv', 'r')
        tick_value = ''
        comapnyName = ''
        Previous_close = ''
        Today_s_open = ''
        Day_s_range = ''
        Volume = ''
        Average_volume = ''
        Market_cap = ''
        data = csv.reader(file_handler)
        for i in data:
            looking_item = i[1]
            if looking_item == user_input:
                tick_value = i[1]
                comapnyName = i[2]
                Previous_close = i[3]
                Today_s_open = i[4]
                Day_s_range = i[5]
                Volume = i[6]
                Average_volume = i[7]
                Market_cap = i[8]
                break

        print("The data for {} {} is the following:".format(tick_value,comapnyName))
        print("{} {}".format(tick_value,comapnyName))
        print("Previous_close:{}".format(Previous_close))
        print("Today_s_open:{}".format(Today_s_open))
        print("Day_s_range:{}".format(Day_s_range))
        print("Volume:{}".format(Volume))
        print("Average_volume:{}".format(Average_volume))
        print("Market_cap:{}".format(Market_cap))
        file_handler.close()

    except:
        print("Can not open The File")






# Fire main function
if __name__ == '__main__':main()








Which stock are you interested in: 

Most Actives :
GE General Electric Co
BAC Bank of America Corp
CCL Carnival Corp
OXY Occidental Petroleum Corp
F Ford Motor Co
WFC Wells Fargo & Co
MRO Marathon Oil Corp
T AT&T Inc
PFE Pfizer Inc
NCLH Norwegian Cruise Line Holdings Ltd



Gainers :
OXY Occidental Petroleum Corp
MRO Marathon Oil Corp
EOG EOG Resources Inc
FTI TechnipFMC PLC
DVN Devon Energy Corp
NOV National Oilwell Varco Inc
PXD Pioneer Natural Resources Co
HAL Halliburton Co
COP ConocoPhillips
CXO Concho Resources Inc



Losers :
D Dominion Energy Inc
SRE Sempra Energy
PHM Pultegroup Inc
CCI Crown Castle International Corp
AWK American Water Works Company Inc
HBI HanesBrands Inc
LEN Lennar Corp
RCL Royal Caribbean Cruises Ltd
CMS CMS Energy Corp
BA Boeing Co


